<a href="https://colab.research.google.com/github/NikitiusIvanov/gbd-life-extension-dashboard/blob/main/preprocessing_all_countries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains preprocessing data for estimation risk factors impact to the life expectancy (https://github.com/NikitiusIvanov/gbd-life-extension-dashboard). 

All data was taken from "Global Burden of Disease Study 2019" results tool (https://vizhub.healthdata.org/gbd-results/).

Jupyter notebook witn calculation impact of risk factors into life expectancy: https://drive.google.com/file/d/1bL0Gejz5ZomiBziLsg3Gk-DPYZagBdI9/view?usp=sharing

For reproduce data manipulation you can load the same data from gbd site or fork it from my google-drive by link: https://drive.google.com/drive/folders/1B7BqeCXWVJunX0cBBFT4d25W_Yaw62Sh?usp=sharing

Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd '/content/drive/MyDrive/gbd-life-expectancy-risk-factors-impact-estimation'

/content/drive/MyDrive/gbd-life-expectancy-risk-factors-impact-estimation


In [3]:
import numpy as np
import pandas as pd
import os
import tqdm

load code book

In [4]:
code_book = pd.read_csv(
    os.path.join(
        'data',
        'coode_book','IHME_GBD_2019_CODEBOOK_Y2022M06D29.CSV'
    )
).iloc[1:, 1:]

code_book.head(3)

,measure_id,measure_name,location_id,location_name,sex_id,sex_label,age_group_id,age_group_name,cause_id,cause_name,rei_id,rei_name,metric_id,metric_name,year_id,val,upper,lower
1,1,Deaths,1,Global,Sex ID,Sex,1,Under 5,294,All causes,169,All risk factors,1,Number,1990,NaN,NaN,NaN
2,2,DALYs (Disability-Adjusted Life Years),44637,Low SDI,1,Male,2,Early Neonatal,295,"Communicable, maternal, neonatal, and nutritio...",202,Environmental/occupational risks,2,Percent,1991,NaN,NaN,NaN
3,3,YLDs (Years Lived with Disability),44636,Low-middle SDI,2,Female,3,Late Neonatal,955,HIV/AIDS and sexually transmitted infections,203,Behavioral risks,3,Rate,1992,NaN,NaN,NaN


In [ ]:
measure_name_to_id = { # create mappings
    key: value 
    for key, value in code_book[['measure_name', 'measure_id']].dropna().values
}

metric_name_to_id = {
    key: value 
    for key, value in code_book[['metric_name', 'metric_id']].dropna().values
}

sex_name_to_id = {
    key: value 
    for key, value in code_book[['sex_label', 'sex_id']].dropna().values
}

age_name_to_id = {
    key: value 
    for key, value in code_book[['age_group_name', 'age_group_id']].dropna().values
}

cause_name_to_id = {
    key: value 
    for key, value in code_book[['cause_name', 'cause_id']].dropna().values
}

risks_name_to_id = {
    key: value 
    for key, value in code_book[['rei_name', 'rei_id']].dropna().values
}

location_name_to_id = {
    key: value 
    for key, value in code_book[['location_name', 'location_id']].dropna().values
}

name_to_id_mappings: dict = {
    'measure_id' :measure_name_to_id,
    'metric_id' :metric_name_to_id,
    'sex_id' :sex_name_to_id,
    'age_id' :age_name_to_id,
    'cause_id' :cause_name_to_id,
    'risks_id' :risks_name_to_id,
    'location_id' :location_name_to_id,
}

Utilite functions

In [ ]:
def find_name_by_fragment_in_id_mapping(
    name_to_id_dict: dict,
    fragment: str
) -> list:
    result = [x for x in name_to_id_dict.keys() if fragment.lower() in x.lower()]
    return result

In [ ]:
def add_names_columns_by_ids(
    df: pd.DataFrame,
    name_to_id_mappings: dict = {
        'measure_id' :measure_name_to_id,
        'metric_id' :metric_name_to_id,
        'sex_id' :sex_name_to_id,
        'age_id' :age_name_to_id,
        'cause_id' :cause_name_to_id,
        'risks_id' :risks_name_to_id,
        'location_id' :location_name_to_id,
    },
) -> pd.DataFrame:
    result = df.copy()
    for column in set(result.columns).intersection(set(name_to_id_mappings.keys())):
        result[column.replace('_id', '_name')] = (
            df[column]
            .astype('str')
            .map({key: value for value, key in name_to_id_mappings[column].items()})
        )
    return result

load causes and risks ierarchy

In [6]:
causes_ierarchy = pd.read_csv(
    os.path.join('data','coode_book','IHME_GBD_2019_CAUSE_HIERARCHY_Y2020M11D25.csv')
)

causes_ierarchy.columns = [x.lower().replace(' ', '_') for x in causes_ierarchy.columns]

In [7]:
causes_ierarchy.head(3)

,cause_id,cause_name,parent_id,parent_name,level,cause_outline,sort_order,yll_only,yld_only
0,294,All causes,294,All causes,0,Total,1,NaN,NaN
1,295,"Communicable, maternal, neonatal, and nutritio...",294,All causes,1,A,2,NaN,NaN
2,955,HIV/AIDS and sexually transmitted infections,295,"Communicable, maternal, neonatal, and nutritio...",2,A.1,3,NaN,NaN


In [8]:
risks_ierarchy = pd.read_csv(
    os.path.join('data', 'coode_book', 'IHME_GBD_2019_REI_HIERARCHY_Y2020M10D15.csv')
)

risks_ierarchy.columns = [x.lower().replace(' ', '_') for x in risks_ierarchy.columns]

risks_ierarchy.head(3)

,rei_id,rei_name,parent_id,parent_name,level,sort_order
0,169,All risk factors,169,All risk factors,0,1.0
1,202,Environmental/occupational risks,169,All risk factors,1,2.0
2,203,Behavioral risks,169,All risk factors,1,3.0


## load mortality (age, sex, cause specific)

In [9]:
causes_all_countries_2019_paths = [x for x in os.walk(os.path.join('data', 'causes'))]

file_paths = []

for folder in causes_all_countries_2019_paths:
    for file_name in folder[2]:
        if '.csv' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data/causes/IHME-GBD_2019_DATA-f7db7ce2-3/IHME-GBD_2019_DATA-f7db7ce2-3.csv',
 'data/causes/IHME-GBD_2019_DATA-f7db7ce2-2/IHME-GBD_2019_DATA-f7db7ce2-2.csv',
 'data/causes/IHME-GBD_2019_DATA-f7db7ce2-1/IHME-GBD_2019_DATA-f7db7ce2-1.csv']

In [ ]:
causes_all_countries_2019 = pd.concat([pd.read_csv(x) for x in file_paths], axis=0)

In [ ]:
causes_all_countries_2019_source = causes_all_countries_2019.copy()

In [ ]:
causes_all_countries_2019.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'metric_id',
       'metric_name', 'year', 'val', 'upper', 'lower'],
      dtype='object')

In [ ]:
for column in causes_all_countries_2019.columns:
    print(column, causes_all_countries_2019[column].unique().shape[0])

measure_id 1
measure_name 1
location_id 204
location_name 204
sex_id 2
sex_name 2
age_id 21
age_name 21
cause_id 134
cause_name 134
metric_id 1
metric_name 1
year 1
val 941970
upper 941320
lower 909977


In [ ]:
causes_all_countries_2019.cause_name[causes_all_countries_2019.cause_name.str.contains('All')].unique()

array(['All causes'], dtype=object)

In [ ]:
# del the sumarry term
causes_all_countries_2019 = causes_all_countries_2019.copy().query('cause_name != "All causes"')

## load risks factors attributed mortality (age, sex, cause, risk specific)

In [10]:
risk_all_countries_2019_paths = [x for x in os.walk(os.path.join('data', 'risk_factors'))]

file_paths = []

for folder in risk_all_countries_2019_paths:
    for file_name in folder[2]:
        if '.csv' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-5/IHME-GBD_2019_DATA-53fb5bb6-5.csv',
 'data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-2/IHME-GBD_2019_DATA-53fb5bb6-2.csv',
 'data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-4/IHME-GBD_2019_DATA-53fb5bb6-4.csv',
 'data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-1/IHME-GBD_2019_DATA-53fb5bb6-1.csv',
 'data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-6/IHME-GBD_2019_DATA-53fb5bb6-6.csv',
 'data/risk_factors/IHME-GBD_2019_DATA-53fb5bb6-3/IHME-GBD_2019_DATA-53fb5bb6-3.csv']

In [ ]:
risk_all_countries_2019 = pd.concat([pd.read_csv(x) for x in file_paths], axis=0)

In [ ]:
risk_all_countries_2019_source = risk_all_countries_2019.copy()

In [ ]:
risk_all_countries_2019.shape

(2792428, 18)

In [ ]:
risk_all_countries_2019.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'rei_id',
       'rei_name', 'metric_id', 'metric_name', 'year', 'val', 'upper',
       'lower'],
      dtype='object')

In [ ]:
for column in risk_all_countries_2019.columns:
    print(column, risk_all_countries_2019[column].unique().shape[0])

measure_id 1
measure_name 1
location_id 204
location_name 204
sex_id 2
sex_name 2
age_id 21
age_name 21
cause_id 80
cause_name 80
rei_id 67
rei_name 67
metric_id 1
metric_name 1
year 1
val 2381251
upper 2383214
lower 2376493


In [ ]:
risk_all_countries_2019.cause_name[risk_all_countries_2019.cause_name.str.contains('All')].unique()

array(['All causes'], dtype=object)

In [ ]:
risk_all_countries_2019 = risk_all_countries_2019.copy().query('cause_name != "All causes"')

## Load life expectancy

In [11]:
life_tables_paths = [x for x in os.walk(os.path.join('data', 'life_tables'))]

life_tables_df = []

for path in life_tables_paths[1:]:
    file_name = next(filter(lambda x: 'WSHOCK' in x, path[2])) #return first path with 'NOSHOCK' in it
    life_tables_df.append(
        pd.read_csv(
            os.path.join(path[0], file_name)
        )
    )
    print(file_name)

IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_10_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_13_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_6_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_12_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_8_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_7_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_9_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_14_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_5_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_11_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_18_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_16_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_28_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_20_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950_2019_ID_30_WSHOCK_Y2020M11D13.CSV
IHME_GBD_2019_LIFE_TABLES_1950

In [ ]:
life_tables = pd.concat(life_tables_df)

print(life_tables.shape)
display(life_tables.head())

(6804000, 14)


,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
0,1,Global,1,male,10,25 to 29,1950,27,Probability of death,8,Probability of death,0.020009,0.021030,0.019039
1,1,Global,1,male,10,25 to 29,1950,26,Life expectancy,5,Years,39.693623,40.543689,38.834807
2,1,Global,2,female,10,25 to 29,1950,27,Probability of death,8,Probability of death,0.018781,0.020751,0.016899
3,1,Global,2,female,10,25 to 29,1950,26,Life expectancy,5,Years,44.072916,44.957524,43.163672
4,1,Global,3,both,10,25 to 29,1950,27,Probability of death,8,Probability of death,0.019387,0.020521,0.018226


In [ ]:
life_expectancy = life_tables.copy().query(
    'measure_name == "Life expectancy"'
    ' and metric_name == "Years"'
    ' and year_id == 2019'
    ' and sex_name != "both"'
)

life_expectancy

,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
415,1,Global,1,male,10,25 to 29,2019,26,Life expectancy,5,Years,49.847937,50.595783,49.155354
417,1,Global,2,female,10,25 to 29,2019,26,Life expectancy,5,Years,54.660094,55.299612,54.035491
835,4,"Southeast Asia, East Asia, and Oceania",1,male,10,25 to 29,2019,26,Life expectancy,5,Years,50.143905,51.609924,48.774651
837,4,"Southeast Asia, East Asia, and Oceania",2,female,10,25 to 29,2019,26,Life expectancy,5,Years,55.661647,56.896661,54.433171
1255,5,East Asia,1,male,10,25 to 29,2019,26,Life expectancy,5,Years,50.938034,52.841619,49.069084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282657,44790,Bournemouth,2,female,9,20 to 24,2019,26,Life expectancy,5,Years,62.391643,63.838374,60.957610
283075,44791,Cornwall,1,male,9,20 to 24,2019,26,Life expectancy,5,Years,60.261833,61.860788,58.747535
283077,44791,Cornwall,2,female,9,20 to 24,2019,26,Life expectancy,5,Years,63.780595,65.032639,62.448782
283495,44792,Plymouth,1,male,9,20 to 24,2019,26,Life expectancy,5,Years,59.053481,60.693160,57.404585


In [ ]:
life_expectancy = life_expectancy.query('location_id in @risk_all_countries_2019.location_id.unique()')

In [ ]:
life_expectancy.age_group_name.unique()

array(['25 to 29', '30 to 34', '35 to 39', '40 to 44', '110 plus',
       '45 to 49', '50 to 54', '55 to 59', '60 to 64', '65 to 69',
       '70 to 74', '75 to 79', '<1 year', '80 to 84', '85 to 89',
       '90 to 94', '95 to 99', '100 to 104', '105 to 109', '1 to 4',
       '5 to 9', '10 to 14', '15 to 19', '20 to 24'], dtype=object)

## load population

In [12]:
population = pd.read_csv(
    os.path.join('data', 'population', 'IHME_GBD_2019_POP_2019_Y2020M10D15.CSV')
)

print(population.shape)
display(population.head())

(95175, 14)


,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
0,1,Global,1,male,1,Under 5,2019,44,Population,1,Number,3.423987e+08,3.523971e+08,3.324841e+08
1,1,Global,1,male,2,Early Neonatal,2019,44,Population,1,Number,1.330245e+06,1.368039e+06,1.292459e+06
2,1,Global,1,male,3,Late Neonatal,2019,44,Population,1,Number,3.951181e+06,4.063467e+06,3.839020e+06
3,1,Global,1,male,4,Post Neonatal,2019,44,Population,1,Number,6.292360e+07,6.472045e+07,6.113298e+07
4,1,Global,1,male,5,1 to 4,2019,44,Population,1,Number,2.741937e+08,2.822446e+08,2.662198e+08


In [ ]:
population.columns

Index(['location_id', 'location_name', 'sex_id', 'sex_name', 'age_group_id',
       'age_group_name', 'year_id', 'measure_id', 'measure_name', 'metric_id',
       'metric_name', 'val', 'upper', 'lower'],
      dtype='object')

In [ ]:
population = population.query('location_id in @risk_all_countries_2019.location_id.unique()')

population preprocessing

In [ ]:
for column in population.columns:
    print(column)
    display(population[column].unique())

location_id


array([    1,     4,     5,     6,     7,     8,     9,    10,    11,
          12,    13,    14,    15,    16,    17,    18,    19,    20,
          21,    22,    23,    24,    25,    26,    27,    28,    29,
          30,    31,    32,    33,    34,    35,    36,    37,    38,
          39,    40,    41,    42,    43,    44,    45,    46,    47,
          48,    49,    50,    51,    52,    53,    54,    55,    56,
          57,    58,    59,    60,    61,    62,    63,    64,    65,
          66,    67,    68,    69,    70,    71,    72,    73,    74,
          75,    76,    77,    78,    79,    80,    81,    82,    83,
          84,    85,    86,    87,    88,    89,    90,    91,    92,
          93,    94,    95,    96,    97,    98,    99,   100,   101,
         102,   103,   104,   105,   106,   107,   108,   109,   110,
         111,   112,   113,   114,   115,   116,   117,   118,   119,
         120,   121,   122,   123,   124,   125,   126,   127,   128,
         129,   130,

location_name


array(['Global', 'Southeast Asia, East Asia, and Oceania', 'East Asia',
       'China', "Democratic People's Republic of Korea",
       'Taiwan (Province of China)', 'Southeast Asia', 'Cambodia',
       'Indonesia', "Lao People's Democratic Republic", 'Malaysia',
       'Maldives', 'Myanmar', 'Philippines', 'Sri Lanka', 'Thailand',
       'Timor-Leste', 'Viet Nam', 'Oceania', 'Fiji', 'Kiribati',
       'Marshall Islands', 'Micronesia (Federated States of)',
       'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Vanuatu',
       'Central Europe, Eastern Europe, and Central Asia', 'Central Asia',
       'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Kyrgyzstan',
       'Mongolia', 'Tajikistan', 'Turkmenistan', 'Uzbekistan',
       'Central Europe', 'Albania', 'Bosnia and Herzegovina', 'Bulgaria',
       'Croatia', 'Czechia', 'Hungary', 'North Macedonia', 'Montenegro',
       'Poland', 'Romania', 'Serbia', 'Slovakia', 'Slovenia',
       'Eastern Europe', 'Belarus', 'Estonia'

sex_id


...

array([3.36538477e+08, 1.34830096e+06, 3.99385814e+06, ...,
       1.49859843e+05, 1.41944147e+05, 2.91803990e+05])

upper


array([3.42899131e+08, 1.37372147e+06, 4.06927487e+06, ...,
       1.64273650e+05, 1.55607486e+05, 3.19881136e+05])

lower


array([3.29991007e+08, 1.32250311e+06, 3.91773029e+06, ...,
       1.36992870e+05, 1.29748176e+05, 2.66741046e+05])

## Data harmonization

deaths and risks preprocessing (sex, metric, measure filtering)

In [ ]:
for name, df in zip(
    ['deaths_by_causes', 'risk_factors'],
    [causes_all_countries_2019, risk_all_countries_2019]
):
    for column in ['sex_id', 'metric_id', 'measure_id']:
        print(
            name, column,
            [
                {k:v for v, k in name_to_id_mappings[column].items()}[str(x)]
                for x in df[column].unique()
            ]
        )

deaths_by_causes sex_id ['Male', 'Female']
deaths_by_causes metric_id ['Number']
deaths_by_causes measure_id ['Deaths']
risk_factors sex_id ['Male', 'Female']
risk_factors metric_id ['Number']
risk_factors measure_id ['Deaths']


age groups names preprocessing (convert to start age)

In [ ]:
code_book.age_group_name.unique()

array(['Under 5', 'Early Neonatal', 'Late Neonatal', 'Post Neonatal',
       '1 to 4', '5 to 9', '10 to 14', '15 to 19', '20 to 24', '25 to 29',
       '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', '65 to 69', '70 to 74', '75 to 79',
       '80 plus', 'All Ages', '5-14 years', '15-49 years', '50-69 years',
       '70+ years', 'Age-standardized', '<1 year', '80 to 84', '85 to 89',
       '90 to 94', '20 plus', '0 to 14', '50 to 74 years', 'Neonatal',
       '25 plus', '<20 years', '10 to 24', '85 plus', 'Birth', '10 to 54',
       '0 to 9', '5 to 19', '15 to 39', '25 to 49', '55 plus', '60 to 79',
       '65 to 74', '75 plus', '95 plus', '75 to 84', '20 to 54 years',
       '55 to 89 years', '60 to 89 years', '65 to 89 years',
       '70 to 89 years', '75 to 94 years', '<70 years', nan], dtype=object)

In [ ]:
causes_all_countries_2019.age_name.unique()

array(['1-4 years', '5-9 years', '10-14 years', '15-19 years',
       '20-24 years', '25-29 years', '30-34 years', '35-39 years',
       '40-44 years', '45-49 years', '50-54 years', '55-59 years',
       '60-64 years', '65-69 years', '70-74 years', '75-79 years',
       '<1 year', '80-84 years', '85-89 years', '90-94 years',
       '95+ years'], dtype=object)

In [ ]:
population.age_group_name.unique()

array(['Under 5', 'Early Neonatal', 'Late Neonatal', 'Post Neonatal',
       '1 to 4', '5 to 9', '10 to 14', '15 to 19', '20 to 24', '25 to 29',
       '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', '65 to 69', '70 to 74', '75 to 79',
       '80 plus', 'All Ages', '5-14 years', '15-49 years', '50-69 years',
       '70+ years', '<1 year', '80 to 84', '85 to 89', '90 to 94',
       '0 to 14', 'Neonatal', '25 plus', '<20 years', '10 to 24',
       '10 to 19', 'Birth', '95 plus', '5 to 19', '15 to 39', '25 to 49',
       '65 to 74', '75 to 84', '20 plus', '55 plus', '75 plus', '85 plus'],
      dtype=object)

In [ ]:
risk_all_countries_2019.age_name.unique()

array(['25-29 years', '30-34 years', '35-39 years', '40-44 years',
       '45-49 years', '50-54 years', '55-59 years', '60-64 years',
       '65-69 years', '70-74 years', '75-79 years', '80-84 years',
       '85-89 years', '90-94 years', '95+ years', '10-14 years',
       '15-19 years', '20-24 years', '<1 year', '1-4 years', '5-9 years'],
      dtype=object)

In [ ]:
(
    risk_all_countries_2019.age_id.astype('str')
    .map({key: value for value, key in age_name_to_id.items()})
    .unique()
)

array(['25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49',
       '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74',
       '75 to 79', '80 to 84', '85 to 89', '90 to 94', '95 plus',
       '10 to 14', '15 to 19', '20 to 24', '<1 year', '1 to 4', '5 to 9'],
      dtype=object)

In [ ]:
age_group_name_to_start_age_int = {
    '<1 year': 0,
    '1 to 4': 1,
    '5 to 9': 5,
    '10 to 14': 10,
    '15 to 19': 15,
    '20 to 24': 20,
    '25 to 29': 25,
    '30 to 34': 30,
    '35 to 39': 35,
    '40 to 44': 40,
    '45 to 49': 45,
    '50 to 54': 50,
    '55 to 59': 55,
    '60 to 64': 60,
    '65 to 69': 65,
    '70 to 74': 70,
    '75 to 79': 75,    
    '80 to 84': 80,
    '85 to 89': 85,
    '90 to 94': 90,
    '95 plus': 95,
}

age_group_id_to_start_age_int = {
    age_name_to_id[key]: value for key, value in age_group_name_to_start_age_int.items() 
}

age_group_id_to_start_age_int

{'28': 0,
 '5': 1,
 '6': 5,
 '7': 10,
 '8': 15,
 '9': 20,
 '10': 25,
 '11': 30,
 '12': 35,
 '13': 40,
 '14': 45,
 '15': 50,
 '16': 55,
 '17': 60,
 '18': 65,
 '19': 70,
 '20': 75,
 '30': 80,
 '31': 85,
 '32': 90,
 '235': 95}

In [ ]:
def age_group_names_preprocessing(
    df: pd.DataFrame,
    age_group_id_column_name: str,
    age_group_id_to_start_age_int: dict={
        '28': 0,
        '5': 1,
        '6': 5,
        '7': 10,
        '8': 15,
        '9': 20,
        '10': 25,
        '11': 30,
        '12': 35,
        '13': 40,
        '14': 45,
        '15': 50,
        '16': 55,
        '17': 60,
        '18': 65,
        '19': 70,
        '20': 75,
        '30': 80,
        '31': 85,
        '32': 90,
        '235': 95
    },
) -> pd.DataFrame:
    result = df.query(
        f'{age_group_id_column_name}.astype("str") in @age_group_id_to_start_age_int.keys()'
    ).copy()

    result['age_group_start'] = (
        result[age_group_id_column_name]
        .astype('str').map(age_group_id_to_start_age_int)
    )

    return result

In [ ]:
risk_all_countries_2019 = age_group_names_preprocessing(
    df=risk_all_countries_2019,
    age_group_id_column_name='age_id'
).copy()

causes_all_countries_2019 = age_group_names_preprocessing(
    df=causes_all_countries_2019,
    age_group_id_column_name='age_id'
).copy()

population = age_group_names_preprocessing(
    df=population,
    age_group_id_column_name='age_group_id'
).copy()

life_expectancy = age_group_names_preprocessing(
    df=life_expectancy,
    age_group_id_column_name='age_group_id'
).copy()

In [ ]:
causes_all_countries_2019.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'metric_id',
       'metric_name', 'year', 'val', 'upper', 'lower', 'age_group_start'],
      dtype='object')

## Data filtering

causes of deaths select detalization level

In [ ]:
causes_names_level_3 = causes_ierarchy.query('level == 3').cause_name.unique()

In [ ]:
print(len(causes_names_level_3))
[np.random.choice(causes_names_level_3, 30)]

169


[array(['Rheumatoid arthritis', 'Cardiomyopathy and myocarditis',
        'Chronic kidney disease', 'Urticaria',
        'Other cardiovascular and circulatory diseases', 'Road injuries',
        'Dengue', 'Self-harm', 'Endocarditis', 'Diabetes mellitus',
        'Larynx cancer', 'Guinea worm disease', 'Non-melanoma skin cancer',
        'Stomach cancer', 'Guinea worm disease',
        'Brain and central nervous system cancer',
        'Interpersonal violence', 'Inflammatory bowel disease',
        'Anxiety disorders', 'Food-borne trematodiases',
        'Multiple sclerosis', 'Esophageal cancer',
        'Dietary iron deficiency', 'HIV/AIDS', 'Oral disorders',
        'Fungal skin diseases', 'Other skin and subcutaneous diseases',
        'Alcohol use disorders',
        'Paralytic ileus and intestinal obstruction', 'Diarrheal diseases'],
       dtype=object)]

In [ ]:
causes_id_level_3 = [int(cause_name_to_id[x]) for x in causes_names_level_3]
len(causes_id_level_3)

169

In [ ]:
causes_all_countries_2019_preprocessed = causes_all_countries_2019.query('cause_id in @causes_id_level_3')

for column in ['location_name', 'sex_name', 'age_name', 'measure_name', 'metric_name', 'year']:

    print(column)
    display(causes_all_countries_2019_preprocessed[column].unique())

location_name


array(['Vanuatu', 'Switzerland', 'Colombia', 'Costa Rica', 'Uganda',
       'Russian Federation', 'Maldives', 'Myanmar', 'Denmark', 'Croatia',
       'Egypt', 'Yemen', 'Cuba', 'Dominica', 'Armenia', 'United Kingdom',
       'Gabon', 'Ukraine', 'Czechia', 'Finland', 'Zambia', 'Gambia',
       'Iran (Islamic Republic of)', 'Puerto Rico',
       'Saint Kitts and Nevis', 'El Salvador', 'Dominican Republic',
       "Côte d'Ivoire", 'Sweden', 'Azerbaijan', 'Sudan', 'American Samoa',
       'Philippines', 'Iraq', 'Argentina', 'Sri Lanka', 'Afghanistan',
       'France', 'Burundi', 'Hungary', 'North Macedonia', 'San Marino',
       'Ghana', 'Bermuda', 'Brunei Darussalam', 'Guinea', 'Guatemala',
       'Jordan', 'Grenada', 'Georgia', 'China', 'Honduras', 'Thailand',
       'Germany', 'Botswana', 'Tokelau', 'Greece', 'Guyana', 'Haiti',
       'Bhutan', 'Uruguay', 'Liberia', 'Comoros', 'Lesotho', 'Kyrgyzstan',
       'Bangladesh', 'Montenegro', 'Greenland', 'Chile', 'Poland',
       'Iceland', "D

sex_name


array(['Male', 'Female'], dtype=object)

age_name


...

array(['Deaths'], dtype=object)

metric_name


array(['Number'], dtype=object)

year


array([2019], dtype=int64)

risk factors most detailed select

In [ ]:
risks_ierarchy.query('level == 0')

,rei_id,rei_name,parent_id,parent_name,level,sort_order
0,169,All risk factors,169,All risk factors,0,1.0
4,171,Etiologies,171,Etiologies,0,5.0
5,191,Impairments,191,Impairments,0,6.0
6,362,Injuries,362,Injuries,0,7.0
146,298,Amputations,298,Amputations,0,148.0
154,299,Burns,299,Burns,0,156.0
158,300,Fractures,300,Fractures,0,160.0
171,301,Head Injuries,301,Head Injuries,0,173.0
174,302,Spinal Injuries,302,Spinal Injuries,0,176.0
177,303,Minor Injuries,303,Minor Injuries,0,179.0


In [ ]:
risks_most_detailed_ids = set(
    risks_ierarchy.rei_id.values
).difference(set(risks_ierarchy.parent_id.values)).union(set([169]))

len(risks_most_detailed_ids)

165

In [ ]:
# select only childrens from 'All risk factors'
start_id = 169
start_level = risks_ierarchy.query('rei_id == @start_id').level.values[0] + 1
all_risks_ids = set([start_id])
for level in range(start_level, risks_ierarchy.level.max() + 1):
    all_risks_ids = all_risks_ids.union(
        set(risks_ierarchy.query('level == @level and parent_id in @all_risks_ids').rei_id.values)
    )

    print(level)
all_risks_ids = all_risks_ids.union(set([169]))
print(len(all_risks_ids))

1
2
3
4
87


In [ ]:
# select most detailed risks that are childrens from 'All risk factors'
risks_most_detailed = risks_ierarchy.query('rei_id in @all_risks_ids and rei_id in @risks_most_detailed_ids')

risks_most_detailed

,rei_id,rei_name,parent_id,parent_name,level,sort_order
0,169,All risk factors,169,All risk factors,0,1.0
8,83,Unsafe water source,82,"Unsafe water, sanitation, and handwashing",3,9.0
9,84,Unsafe sanitation,82,"Unsafe water, sanitation, and handwashing",3,10.0
10,238,No access to handwashing facility,82,"Unsafe water, sanitation, and handwashing",3,11.0
13,86,Ambient particulate matter pollution,380,Particulate matter pollution,4,14.0
...,...,...,...,...,...,...
85,367,High LDL cholesterol,104,Metabolic risks,2,85.0
86,107,High systolic blood pressure,104,Metabolic risks,2,86.0
87,108,High body-mass index,104,Metabolic risks,2,87.0
88,109,Low bone mineral density,104,Metabolic risks,2,88.0


In [ ]:
set(risks_most_detailed.rei_id).intersection(set(risks_most_detailed.parent_id))

{169}

In [ ]:
len(risk_all_countries_2019.rei_id.unique())

67

In [ ]:
risk_all_countries_2019_preprocessed = risk_all_countries_2019.query(
    'rei_id in @risks_most_detailed.rei_id'
    ' and cause_name in @causes_names_level_3'
)

len(risk_all_countries_2019_preprocessed.rei_id.unique())

67

In [ ]:
population = population.copy().query(
    'sex_name != "both"'
    ' and age_group_start in @causes_all_countries_2019.age_group_start'
    ' and year_id == 2019'
    ' and location_name in @causes_all_countries_2019.location_name'
)

In [ ]:
life_expectancy = life_expectancy.copy().query(
    'sex_name != "both"'
    ' and age_group_start in @causes_all_countries_2019.age_group_start'
    ' and year_id == 2019'
    ' and location_name in @causes_all_countries_2019.location_name'
)

## Filling risk factors

In [ ]:
risk_all_countries_2019_preprocessed.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'rei_id',
       'rei_name', 'metric_id', 'metric_name', 'year', 'val', 'upper', 'lower',
       'age_group_start'],
      dtype='object')

In [ ]:
risk_all_countries_2019_preprocessed.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'cause_id', 'cause_name', 'rei_id',
       'rei_name', 'metric_id', 'metric_name', 'year', 'val', 'upper', 'lower',
       'age_group_start'],
      dtype='object')

In [ ]:
results = []

for cause_id in tqdm.tqdm(risk_all_countries_2019_preprocessed['cause_id'].unique()):

    for rei_id in risk_all_countries_2019_preprocessed.query(
        'cause_id == @cause_id'
    )['rei_id'].unique():

        filled_risk_factor = (
            risk_all_countries_2019_preprocessed.query(
                'cause_id == @cause_id'
                ' and rei_id == @rei_id'
            ).set_index(
                [
                    'location_id',
                    'age_group_start',
                    'sex_id',
                    'cause_id',
                    'rei_id'
                ]
            )[['val', 'upper', 'lower']].sort_index()
        ).join(
            pd.DataFrame(
                index=pd.MultiIndex.from_product(
                    [
                        risk_all_countries_2019_preprocessed['location_id'].unique(),
                        risk_all_countries_2019_preprocessed['age_group_start'].unique(),
                        risk_all_countries_2019_preprocessed['sex_id'].unique(),
                        [cause_id],
                        [rei_id]
                    ],
                    names=[
                        'location_id',
                        'age_group_start',
                        'sex_id',
                        'cause_id',
                        'rei_id'
                    ]
                )).sort_index(),
            how='outer'    
        ).fillna(0).reset_index()

        results.append(filled_risk_factor)

100%|██████████| 79/79 [00:45<00:00,  1.73it/s]


In [ ]:
filled_risk_factors = pd.concat(results, axis=0)

In [ ]:
filled_risk_factors.shape

(3581424, 8)

In [ ]:
filled_risk_factors.duplicated().sum()

0

In [ ]:
(filled_risk_factors[['val', 'upper', 'lower']] == 0).all(axis=1).sum()

1256614

In [ ]:
filled_risk_factors

,location_id,age_group_start,sex_id,cause_id,rei_id,val,upper,lower
0,6,0,1,504,91,0.000000,0.000000,0.000000
1,6,0,2,504,91,0.000000,0.000000,0.000000
2,6,1,1,504,91,0.000000,0.000000,0.000000
3,6,1,2,504,91,0.000000,0.000000,0.000000
4,6,5,1,504,91,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
8563,522,85,2,499,124,0.052622,0.187631,0.012720
8564,522,90,1,499,124,0.061035,0.292564,0.009096
8565,522,90,2,499,124,0.027368,0.100173,0.006522
8566,522,95,1,499,124,0.022894,0.103315,0.003498


In [ ]:
risks_name_to_id = {k: int(v) for k,v in risks_name_to_id.items()}
location_name_to_id = {k: int(v) for k,v in location_name_to_id.items()}
cause_name_to_id = {k: int(v) for k,v in cause_name_to_id.items()}

In [ ]:
filled_risk_factors.query(
    'rei_id == @risks_name_to_id["Iron deficiency"]'
    ' and location_id == @location_name_to_id["Germany"]'
).val.sum()

3.1893792590535663

In [ ]:
filled_risk_factors.to_csv(
    os.path.join('data', 'population_causes_risks_preprocessed', 'risk_factors.csv'),
    index=False,
)

In [ ]:
pd.Series(risks_name_to_id).to_csv(
    os.path.join('data', 'population_causes_risks_preprocessed', 'map_risks_name_to_id.csv'),
)

pd.Series(location_name_to_id).to_csv(
    os.path.join('data', 'population_causes_risks_preprocessed', 'map_location_name_to_id.csv'),
)

pd.Series(cause_name_to_id).to_csv(
    os.path.join('data', 'population_causes_risks_preprocessed', 'map_cause_name_to_id.csv'),
)

In [ ]:
filled_risk_factors.rei_id.unique().shape

(67,)

## Selecting risk factors that might be excluded

In [ ]:
start_id = 203 # rei_name = 'Behavioral risks'
start_level = risks_ierarchy.query('rei_id == @start_id').level.values[0] + 1
behavioral_risks_ids = set([start_id])
for level in range(start_level, risks_ierarchy.level.max() + 1):
    behavioral_risks_ids = behavioral_risks_ids.union(
        set(risks_ierarchy.query('level == @level and parent_id in @behavioral_risks_ids').rei_id.values)
    )

    print(level)

2
3
4


In [ ]:
behavioral_risks = risks_ierarchy.query('rei_id in @behavioral_risks_ids')
len(behavioral_risks)

43

In [ ]:
behavioral_risks

,rei_id,rei_name,parent_id,parent_name,level,sort_order
2,203,Behavioral risks,169,All risk factors,1,3.0
42,92,Child and maternal malnutrition,203,Behavioral risks,2,43.0
43,93,Suboptimal breastfeeding,92,Child and maternal malnutrition,3,44.0
44,136,Non-exclusive breastfeeding,93,Suboptimal breastfeeding,4,45.0
45,137,Discontinued breastfeeding,93,Suboptimal breastfeeding,4,46.0
46,239,Child growth failure,92,Child and maternal malnutrition,3,47.0
47,94,Child underweight,239,Child growth failure,4,48.0
48,240,Child wasting,239,Child growth failure,4,49.0
49,241,Child stunting,239,Child growth failure,4,50.0
50,339,Low birth weight and short gestation,92,Child and maternal malnutrition,3,51.0


In [ ]:
risks_ierarchy[risks_ierarchy.rei_name.str.contains('High')]

,rei_id,rei_name,parent_id,parent_name,level,sort_order
17,337,High temperature,331,Non-optimal temperature,3,18.0
84,105,High fasting plasma glucose,104,Metabolic risks,2,84.0
85,367,High LDL cholesterol,104,Metabolic risks,2,85.0
86,107,High systolic blood pressure,104,Metabolic risks,2,86.0
87,108,High body-mass index,104,Metabolic risks,2,87.0


In [ ]:
risks_manageable = [
    'Alcohol use',
    'Drug use',
    'Smoking',
    'Secondhand smoke',
    'Chewing tobacco',
    'Unsafe sex',
    'Low physical activity',
    'High fasting plasma glucose',
    'High LDL cholesterol',
    'High systolic blood pressure',
    'High body-mass index',
    'Diet high in processed meat',
    'Diet high in red meat',
    'Diet high in sodium',
    'Diet high in sugar-sweetened beverages',
    'Diet high in trans fatty acids',
    'Diet low in calcium',
    'Diet low in fiber',
    'Diet low in fruits',
    'Diet low in legumes',
    'Diet low in milk',
    'Diet low in nuts and seeds',
    'Diet low in polyunsaturated fatty acids',
    'Diet low in seafood omega-3 fatty acids',
    'Diet low in vegetables',
    'Diet low in whole grains',    
]

In [ ]:
len(risks_manageable)

26

In [ ]:
pd.Series(risks_manageable).to_csv(
    os.path.join(
        'data',
        'population_causes_risks_preprocessed',
        'risks_names_manageable.csv'
    ),
    index=False
)

In [ ]:
map_rei_id_to_rei_parent_id = {k:v for k,v in risks_ierarchy[['rei_id', 'parent_id']].values}
map_cause_id_to_cause_parent_id = {k:v for k,v in causes_ierarchy[['cause_id', 'parent_id']].values}

In [ ]:
pd.Series(map_rei_id_to_rei_parent_id).to_csv(
    os.path.join(
        'data',
        'population_causes_risks_preprocessed',
        'map_rei_id_to_rei_parent_id.csv'
    ),
)

pd.Series(map_cause_id_to_cause_parent_id).to_csv(
    os.path.join(
        'data',
        'population_causes_risks_preprocessed',
        'map_cause_id_to_cause_parent_id.csv'
    ),
)

## Save preprocessed data

In [ ]:
preprocessed_path = os.path.join('data', 'data_preprocessed')

In [ ]:
population.to_csv(
    os.path.join(
        preprocessed_path,
        'population.csv'
    ),
    index=False
)

In [ ]:
causes_all_countries_2019_preprocessed.to_csv(
    os.path.join(
        preprocessed_path,
        'causes.csv'
    ),
    index=False
)

In [ ]:
filled_risk_factors.to_csv(
    os.path.join(
        preprocessed_path,
        'risk_factors.csv'
    ),
    index=False
)

In [ ]:
life_expectancy.to_csv(
        os.path.join(
        preprocessed_path,
        'life_expectancy.csv'
    ),
    index=False
)